# Exact emulator for  Two-Body scattering

This notebook contains the implementation of two emulators (Woodbury-identity based and projection based) for two-body scattering.
The goal is to find solutions to the Lippmann-Schwinger equation (LSE), at fixed scattering energy for any set of parameters $c_1$, $c_2$, $c_3$ which enter the potential.

For exact and low-cost emulation, three conditions must be fulfilled:

<ul>

<li>

Fixed energy $E_\text{cms}$
</li>

<li>

Potential in the affine form (i.e. $V(p',p)=V_0(p',p)+c_1V_1(p',p)+c_2V_2(p',p)+c_3V_3(p',p)$)
</li>

<li>

Parameter-dependent parts of potential ($V_1$,$V_2$,$V_3$) should be low-rank. Parameter-independent part $V_0$ may have any rank.
</li>

</ul>

This notebook demonstrates, that our emulators provide exact and low-computational-cost solutions of the Lippmann-Schwinger equation.

## Table of contents
<ul>
<li> Definition of Lippmann-Schwinger equation and toy-model potential </li>
<li> Numerical setup<ul>
<li> Physical constants, kinematics and integration grids</li>
<li> Solving Lippmann-Schwinger equation numerically using subtraction technique</li>
<li> Discretized potential</li>
</ul></li>
<li> Direct numerical solution of the Lippmann-Schwinger equation for specific parameter values </li>
<li> Emulator based on Woodbury identity for Lippmann-Schwinger equation (and comparison to direct solution) </li>
<li> Emulator based on snapshots and projection on reduced basis (and comparison to direct solution) </li>
<li> Comparison of the emulators </li>
</ul>

## Definition of Lippmann-Schwinger equation and toy model potential

As a toy-model of two-body (NN) scattering, we consider spinless particles and only $S$-wave scattering. Calculations are done in momentum space.

### Lippmann-Schwinger equation
The Lippmann-Schwinger equation (LSE) allows one to compute the on-shell elements of the K-matrix, which determines observables like the phase shift.
We consider the LSE in partial-wave basis:

$$

K(p',p) = V(p',p) + 2m_{\text{red}} \int_0^\infty \mathrm{d}k\, k^2 \frac{V(p',k)\,K(k,p)}{q^2 - k^2}.

$$

where $K(p',p)$ is the K-matrix, $V(p',p)$ is the potential and $p$, ($p'$) is the relative momentum in the center-of-mass frame before (after) the scattering. $m_{\text{red}}$ is the reduced mass of the two-body system and $q$ is the on-shell momentum.

### Potential 

The potential $V (p', p)$ for which we want to calculate solutions of the Lippmann-Schwinger equation is given by 
$$
\begin{align*}
V (p', p) &= \underbrace{-\frac{\alpha}{8 \pi^2 p' p} \ln \left( \frac{(p'+p)^2 + M^2}{(p'-p)^2 + M^2} \right)}_{V_0}\\
& + c_1\cdot \underbrace{e^{-(p'{}^2 + p^2)/\Lambda^2}}_{V_1} + c_2 \cdot \underbrace{(p'^2 + p^2)e^{-(p'{}^2 + p^2)/\Lambda^2}}_{V_2} + c_3 \cdot \underbrace{p'{}^2p^2 e^{-(p'{}^2 + p^2)/\Lambda^2}}_{V_3}
\end{align*}
$$

$V_0$ is a partial-wave decomposed Yukawa-potential, with mass $M$ of the exchange-particle and strength $\alpha$. $c_1$, $c_2$, $c_3$ are the parameters of the potential and $V_1$, $V_2$ and $V_3$ their corresponding low-rank potentials.

### Phase shift

The on-shell $K$-matrix element $K(q,q)$ is related to the scattering phase shift $\delta$ via

$$

\delta(q) = -\text{arctan}(\pi 2m_\text{red}qK(q,q))

$$

## Numerical setup

### Physical constants, kinematics and integration grids

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy
import scipy
import time

In [2]:
# define units: 
# to enter dimensionful quantities in code, multiply with respective unit
# to extract dimensionful quantities from code, divide by respective unit
GeV            = 1
MeV            = GeV/1e3

reduced_mass    = 470*MeV # neutron-proton reduced mass
mass_pion       = 138*MeV
ga              = 1.29
Fpi             = 0.0924*GeV

In [3]:
# Ecms is the fixed (!) energy, for which we build emulator
# scattering energy in the center-of-mass-frame (cms)
Ecms                = 10*MeV              
# squared on-shell momentum in cms
onshell_momentum_sq = 2*Ecms*reduced_mass 
onshell_momentum    = np.sqrt(onshell_momentum_sq)

### Solving Lippmann-Schwinger equation numerically using subtraction technique

The Lippmann-Schwinger equation in momentum space is an integral equation, which can be discretized and written as a matrix equation.

We solve the Lippmann–Schwinger equation for the $K$-matrix

$$

K(p',p) = V(p',p) + 2m_{\text{red}} \int_0^\infty \mathrm{d}k\, k^2 \frac{V(p',k)\,K(k,p)}{q^2 - k^2}.

$$

The integrand has a pole at $k=q$. We handle it by subtraction

$$
\begin{aligned}
K(p',p) &= V(p',p) \\
&\quad + 2m_{\text{red}} \int_0^\infty \mathrm{d}k\, k^2 
\frac{V(p',k)K(k,p) - V(p',q)K(q,p)}{q^2 - k^2} \\
&\quad + 2m_{\text{red}} V(p',q)K(q,p)
\int_0^\infty \mathrm{d}k\, k^2 \frac{1}{q^2 - k^2}.
\end{aligned}
$$

The first integral is now regular and can be discretized, while the second term can be evaluated analytically.

#### Discretization
Discretizing momentum space integration can be done by the replacement $\int \text{d}k f(k) \rightarrow \sum_i \Delta_i f(k_i)$, where $i=1,...,n$ and $n$ is the total number of grid points used.


Define the matrices
$$
K_{ij} = K(p_i,p_j), \quad V_{ij} = V(p_i,p_j).
$$
We include the on-shell momentum explicitly as an additional grid point, i.e. $k_{n+1} = q$.

#### Matrix equation

For $i,j = 1,\dots, n+1$ we can write 

$$
K_{ij} = V_{ij} + \sum_{l=1}^{n+1} V_{il}\, G_l\, K_{lj},
$$

with the diagonal matrix $G$
$$
G = \mathrm{diag}(G_1, \dots, G_n, G_{n+1}).
$$

The entries of $G$ are

- For $l \le n$:
$$
G_l = \frac{2m_{\text{red}}\, \Delta_l\, p_l^2}{q^2 - p_l^2},
$$

- For $l = n+1$:
$$
G_{n+1} = -\sum_{l=1}^n \frac{2m_{\text{red}}\, \Delta_l\, q^2}{q^2 - p_l^2}
+ 2m_{\text{red}} \frac{1}{2q}
\ln\!\left(\frac{p_{\max}+q}{p_{\max}-q}\right).
$$
In general, the integral in the LSE goes from $0$ to $\infty$, but for simplicity we use large momentum $p_{\max}$ as the upper limit of the integration range.

The final form of the Lippmann-Schwinger equation in matrix form is:
$$
K = V + V G K
$$

The subtraction introduces the on-shell term $K(q,q)$, which is not part of the original $n$ grid points. Therefore, we extend the grid by one point, the on-shell point $q$, all matrices are therefore of size $n+1$. The quantity needed to compute scattering observables is only the on-shell element $K_{n+1, n+1}$.

In [4]:
def grid(a, b, n):
    """
    Returns n Gauss-Legendre mesh points p and weights w on [a, b].
    Input: lower limit of integration interval a,
           upper limit of integration interval b,
           number of mesh points n
    Output: mesh points p and mesh weights w
    """
    x, w = np.polynomial.legendre.leggauss(n)
    p = 0.5*(b-a)*x + 0.5*(b+a)
    w = 0.5*(b-a)*w
    return p, w

In [5]:
# momentum mesh used to discretize integrals
p_max               = 2*GeV # upper end of integration mesh (instead of infinity)
nof_p_points        = 100   # number of points in integration mesh
p_points, p_weights = grid(0, p_max, nof_p_points)

In [6]:
# Here we compute the matrix G

# G_{i,i} elements for i = 1,...,n
G_vec  = 2*reduced_mass*p_weights*p_points**2 / \
        (onshell_momentum_sq-p_points**2)

# G_{n+1, n+1} element
on_q_w = -np.sum(2*reduced_mass*p_weights*onshell_momentum_sq / \
                             (onshell_momentum_sq-p_points**2)) \
                            +2*reduced_mass* \
                            np.log((p_max+onshell_momentum)/ \
                                   (p_max-onshell_momentum))/ \
                                   2*onshell_momentum

# include the on-shell momentum as the n+1 mesh point
p_points = np.append(p_points, onshell_momentum)

# Uncomment to obtain the T-matrix (solving LSE with not only the PV kernel)
# onshellq_weight   += -1j*np.pi*onshell_momentum*reduced_mass
G = np.diag(np.append(G_vec, on_q_w))

## Toy-model potential

The toy-model potential mimics the two-nucleon interaction and consists of two parts: long-range (parameter-free) and short-range (parameter-dependent).

The parameter-free part of the interaction potential $V_0$ is a Yukawa-type potential.
Strength and mass are chosen to be similar to the one-pion-exchange.

For normalization conventions we follow the two-nucleon SMS potential (see arXiv:1711.08821).

In [7]:
pot_norm = 1/(2*np.pi)**3 # normalization convention of the SMS potential
def yukawa(p, pp):
    """
    Toy model Yukawa potential in momentum space projected onto S partial wave. 
    This part of potential is parameter free.
    Mass and prefactor are in close analogy as the one-pion-exchange.
    
    Input:  p (pp) absolute values of initial (final) relative momenta 
    Output: V0 two-body potential
    """   
    alpha  = ( mass_pion * ga / (2*Fpi) )**2
    result = -alpha*2*np.pi/(2*p*pp)*np.log(((p+pp)**2+mass_pion**2)\
                                           /((p-pp)**2+mass_pion**2))
    return pot_norm*result

### Parameter-dependent part of potential

We choose the parameter-dependent potential in a separable form. It is given by

$$ 

\mathrm{exp}(-p'^2/\Lambda^2)\left[c_1+c_2\cdot(p'^2+p^2)+c_3\cdot p'^2 p^2 \right]\mathrm{exp}(-p^2/\Lambda^2)

$$
with the three parameters $c_1$, $c_2$, $c_3$ and the fixed cutoff $\Lambda=500\,\text{MeV}$.

The main goal of emulator is to quickly produce the solutions of LSE for any values of these parameters.

On the discretized momentum mesh this potential is a matrix $V_{ij}$ with the following entries:

$$
V_{ij} = \mathrm{exp}(-p_i^2/\Lambda^2)\left[c_1+c_2\cdot(p_i^2+p_j^2)+c_3\cdot p_i^2 p_j^2 \right]\mathrm{exp}(-p_j^2/\Lambda^2) \\
$$

In [8]:
unit_c1 = 1e4/GeV**2 # Following the SMS convention for units of LECs (parameters)
unit_c2 = 1e4/GeV**4 # in this units LECs are expected to be 
unit_c3 = 1e4/GeV**6 # natural, i.e. roughly of size 1

# regulator function to regularize the potential updates
def regulator(p, Lambda=0.5*GeV):
    """
    Regulator function e^(-p^2/Lambda^2)
    Input:  absolute value of relative momenta (p)
            and cutoff Lambda 
    Output: e^(-p^2/Lambda^2)
    """
    return np.exp(-p**2/Lambda**2)


def potential123(p, pp, c1,c2,c3):
    """
    Gives the value for the potential c1*V1+c2*V2+c3*V3
    Input:  p (pp) absolute values of initial (final) relative momenta
            c1,c2,c3 parameters of potential [dimensionless!]
    Output: c1*V1+c2*V2+c3*V3 two-body potential
    """   
    V1 =              regulator(p)*regulator(pp)
    V2 = (p**2+pp**2)*regulator(p)*regulator(pp)
    V3 = p**2*pp**2  *regulator(p)*regulator(pp)

    return pot_norm*(c1*unit_c1*V1+c2*unit_c2*V2+c3*unit_c3*V3)

### Parameter-dependent part of the potential as a low-rank matrix

We can rewrite the parameter-dependent part of the potential as the product of three matrices:

$$
V_{ij} = [X\cdot C\cdot Z]_{ij}
$$

with the the $n \times 2$ matrix $X_{i1}=\mathrm{exp}(-p_i^2/\Lambda^2)$, $X_{i2}=\mathrm{exp}(-p_i^2/\Lambda^2)p_i^2$, 
the $2 \times 2$ matrix $C=[[c1,c2],[c2,c3]]$ and the $ 2 \times n$ matrix $Z=X^T$.

In such form, it is obvious that the parameter-dependent part of potential is a matrix of rank 2.

In [9]:
# three parameters as sympy symbols
c1_symb, c2_symb, c3_symb = sympy.symbols(r'c_1 c_2 c_3')

# matrices which define the low-rank update to the
# parameter-free potential matrix
X          = np.sqrt(pot_norm)*\
             np.array([regulator(p_points),
                       p_points**2*regulator(p_points)]).T
Cmat_symb  = np.array([[c1_symb*unit_c1, c2_symb*unit_c2],
                       [c2_symb*unit_c2, c3_symb*unit_c3]])
Z          = X.T
# full potential is now given by V=V0+X@C@Z

In [10]:
# verify that both methods to calculate parameter-dependent
# potential yield the same values
trial_c1 = 1
trial_c2 = 2
trial_c3 = 3

# Calculate potential in the usual way
V123       = np.zeros((nof_p_points+1, nof_p_points+1))
for p_idx, p in enumerate(p_points):
    for pp_idx, pp in enumerate(p_points):
        V123[p_idx, pp_idx]=potential123(p=p, pp=pp,
                                         c1=trial_c1,
                                         c2=trial_c2,
                                         c3=trial_c3)

# Calculate potential using X@C@Z form
C  = np.array([[trial_c1*unit_c1, trial_c2*unit_c2],
               [trial_c2*unit_c2, trial_c3*unit_c3]])
V123_lr =X@C@Z

print(np.allclose(V123, V123_lr)) # both methods yield same matrix

True


## Direct numerical solution of the Lippmann-Schwinger equation for specific parameter values

In [11]:
# pre-compute the parameter-free Yukawa potential
# for mesh points and store it in the matrix V0
V0 = np.zeros((nof_p_points+1, nof_p_points+1))
for p_idx, p in enumerate(p_points):
    for pp_idx, pp in enumerate(p_points):
        V0[p_idx, pp_idx]=yukawa(p=p, pp=pp)

In [12]:
# Calculate half-shell K-matrices for a given set of parameters
# by solving the matrix eq. (1-VG)K=V for K and measure time

all_c1 = np.linspace(-10,10, 20) # set of parameters c1,c2,c3
all_c2 = np.linspace(-10,10, 20) # which we use for 
all_c3 = np.linspace(-10,10, 20) # benchmarks

n_tests         = len(all_c1)*len(all_c2)*len(all_c3)
all_halfshell_K = np.zeros((n_tests, nof_p_points+1),dtype=G.dtype)
one_n           = np.eye(len(G))

start = time.perf_counter()
idx=0
for c1 in all_c1:
    for c2 in all_c2:
        for c3 in all_c3:
            # compute matrix C for given parameters
            Cmat = np.array([[c1*unit_c1, c2*unit_c2],
                             [c2*unit_c2, c3*unit_c3]])
            # compute full potential V
            V = V0 + X @ Cmat @ Z
            # solve the LSE directly for half-shell K-matrix
            # we obtain only the half-shell elements of K-matrix by 
            # solving for last column of V, i.e. V[:,-1]
            all_halfshell_K[idx]=np.linalg.solve( one_n - V @ G , V[:,-1])
            idx+=1
end = time.perf_counter()
scattering_ordinary_time = end-start

In [13]:
print('Calculating a solution for '+str(n_tests)+
      ' parameter sets took '+(str(round(scattering_ordinary_time,2)))+'s')

Calculating a solution for 8000 parameter sets took 1.54s


## Emulator based on Woodbury identity for Lippmann-Schwinger equation

Woodbury identity for the LSE for $K$-matrix reads:

$$
\begin{align*}
K &=& K_0 + \tilde{X} \tilde{C} \tilde{Z}, \qquad \text{where}
     \\
    \tilde{C} &\equiv& \big[𝟙_r - C Z G (K_0  G + 𝟙_n) X \big]^{-1}  C,
     \\
     \tilde{X} &\equiv& (K_0  G + 𝟙_n)  X
     \;\;\text{  and  }\;\;
     \tilde{Z} \equiv Z (𝟙_n + G  K_0). \nonumber
\end{align*}
$$

and the matrix $K_0$ is the solution of the LSE without the parameter-dependent part of potential.

In [14]:
# Woodbury formula for LSE allows us to compute 
# the on-shell K-matrix for any c1,c2,c3 analytically.

# For this we will need to precompute 
# the full (n+1) x (n+1)  matrix K0.
K0_mat        = np.linalg.solve( one_n - V0 @ G , V0)

# Compute Woodbury formula for LSE
one_r         = np.eye(len(Cmat_symb))
X_tilde       =     (one_n +     K0_mat @ G) @ X
Z_tilde       = Z @ (one_n + G @ K0_mat)
mat_to_inv    = one_r - Cmat_symb @ (Z @ G @ X_tilde)
# invert matrix using sympy, but store as a numpy array
inv_mat       = np.array(sympy.Matrix(mat_to_inv.tolist()).inv())
C_tilde       = inv_mat @ Cmat_symb

K_on_woodbury = K0_mat[-1,-1] + (X_tilde[-1,:] @ C_tilde @ Z_tilde[:,-1])
# Chop negligible prefactors, which are artefact of finite machine precision
K_on_woodbury = sympy.simplify(K_on_woodbury)
K_on_woodbury = K_on_woodbury.xreplace({
    c: 0 for c in K_on_woodbury.atoms(sympy.Float) if abs(c) < 1e-15
    })

In [15]:
K_on_woodbury.evalf(8)

(6.0793927*c_1*c_3 + 40.238511*c_1 - 6.0793927*c_2**2 - 0.62732563*c_2 - 0.23733737*c_3 - 1.5870807)/(1.0500954*c_1*c_3 + 12.817042*c_1 - 1.0500954*c_2**2 + 2.1105308*c_2 + 0.16881277*c_3 + 1.0)

In [16]:
K_on_func = sympy.lambdify([c1_symb, c2_symb, c3_symb], K_on_woodbury, modules='numpy')

# Calculate all on-shell K-matrix elements with the above formula
# and measure time 
c1_, c2_, c3_ = np.meshgrid(all_c1, all_c2, all_c3, indexing='ij')
c1_ = c1_.flatten()
c2_ = c2_.flatten()
c3_ = c3_.flatten()

start = time.perf_counter()
all_onshell_K_viaprojection = K_on_func(c1_, c2_, c3_)
end = time.perf_counter()
scattering_new_time=end-start

In [17]:
# check if emulator result match direct solution
# for on-shell Kmatrix elements
print(np.allclose(all_onshell_K_viaprojection, all_halfshell_K[:,-1]))

True


In [18]:
# Compute how much faster emulator is compared
# to direct solution
round(scattering_ordinary_time/scattering_new_time)

5673

For our setup, speed-up is $\sim 10^3-10^4$.


We reduced the computational problem for obtaining on-shell $K$-matrix elements for different values of $c_1$, $c_2$ and $c_3$ from solving a $101\times101$ matrix equation to computing a ratio of two  poylnomials. We also demonstrated, both methods yield the exact same result.


This demonstrates how Woodbury identity for LSE can be used to build exact emulator.

## Snapshot-based emulation

Instead of directly applying Woodbury identity for LSE, we can also project the LSE on a reduced basis. Such basis can be found using SVD of several direct solutions (these solutions are called snapshots).

In [19]:
# Just as a test:
# Do an SVD of 8000 half-shell K vectors, to check the dimensionality
# of the subspace in which the half-shell K vectors vary due to
# parameter variation.
_,S_allhalfshellk,_ = scipy.linalg.svd(all_halfshell_K)

In [20]:
plt.plot(np.arange(1,len(S_allhalfshellk)+1), S_allhalfshellk/np.max(S_allhalfshellk), 'o', ls='--')
plt.yscale('log')
plt.ylabel(r'$\sigma_i/max_i(\sigma_i)$')
plt.xlabel(r'$i$')
plt.show()

Performing an SVD on the 8000 solutions for different $c_1$, $c_2$, $c_3$ shows that only three singular values are non-zero, which demonstrates that all solutions lie in just a three-dimensional subspace.

This result is fully in agreement with Woodbury identity for LSE, which predicts that $K$-matrix will have $r+1$ independent components, with $r$ as rank of the parameter-dependent part of potential.

Woodbury identity for LSE implies that dimension of subspace of possible solution for our model is exactly 3. Therefore, we can build build an exact emulator by taking at least 3 linearly independent snapshots (exact solutions), orthogonalize them and project on such reduced basis the LSE.

In the following we will construct such an emulator:

In [21]:
# Define 3 points for snapshots.
# Please note, that c3 is zero in all 3 snapshots.
snapshot_params = [[ 0,  0,  0],
                   [.1,  0,  0],
                   [ 0, .1,  0]]

# Calculate and store half-shell K-matrix for c1,c2,c3 values in 
# the array snapshot_params
just_a_few_snaps = np.zeros((len(snapshot_params), nof_p_points+1))
for idx, [c1,c2,c3] in enumerate(snapshot_params):
    Cmat = np.array([[c1*unit_c1, c2*unit_c2],
                        [c2*unit_c2, c3*unit_c3]])
    V = V0 + X @ Cmat @ Z
    just_a_few_snaps[idx]=np.linalg.solve( one_n - V @ G , V[:,-1])

#Do an SVD of just these 3 snapshots
U,S,Vh           = scipy.linalg.svd(just_a_few_snaps, full_matrices=False)

In [22]:
# now compute projection of LSE on reduced basis

# counting how many basis vectors are needed based on
# number of non-zero singular values
basis_size = np.count_nonzero(S/np.max(S)>1e-10)
# build reduced basis from the elements of Vh
B          = Vh[:basis_size, :].conj().T 
# compute B^{dagger}
B_dag      = np.conj(B).T

# compute left-hand-side of LSE as symbolic matrix
lhs  = -(B_dag @ X) @ Cmat_symb @ (Z @ G @ B) 
lhs += -B_dag @ V0 @ G @ B
lhs +=  B_dag @ np.eye(len(G)) @ B
lhs  = sympy.Matrix(lhs.tolist())

# compute right-hand-side of LSE as symbolic matrix
rhs  = ((B_dag @ X ) @ Cmat_symb @ Z)[:,-1]
rhs += (B_dag @ V0)[:,-1]
rhs  = sympy.Matrix(rhs.tolist())

# solve the projected LSE symbolically
alpha_scatt = sympy.simplify( lhs.inv() @ rhs)

In [23]:
# go from reduced basis back to original n+1 dimensional space
K_halfshell_sym = ( B@alpha_scatt )
# extracting on-shell K-matrix element as symbolic expression
K_on_proj = sympy.simplify(K_halfshell_sym[-1,0])
# Chop negligible prefactors, which are artefact of finite machine precision
K_on_proj = K_on_proj.xreplace({
    c: 0 for c in K_on_proj.atoms(sympy.Float) if abs(c) < 1e-14
    })

In [24]:
K_on_proj.evalf(8)

(4.6684661*c_1*c_3 + 30.899818*c_1 - 4.6684661*c_2**2 - 0.48173371*c_2 - 0.18225529*c_3 - 1.2187455)/(0.80638559*c_1*c_3 + 9.8424185*c_1 - 0.80638559*c_2**2 + 1.6207115*c_2 + 0.12963412*c_3 + 0.76791652)

## Comparison of the emulators

Here we verify that semi-analytic formula for $K$-matrix obtained with Woodbury identity for LSE is the same as with the projection on reduced basis approach.

In [25]:
# check if both expressions for on-shell K-matrix element are the same:

is_this_zero = sympy.expand(sympy.simplify(K_on_proj-K_on_woodbury))

# Chop negligible prefactors, which are artefact of finite machine precision
is_this_zero = is_this_zero.xreplace({
    c: 0 for c in is_this_zero.atoms(sympy.Float) if abs(c) < 1e-12
    })

# Indeed, analytic formula using Woodbury identity and projection method yield same
# semi-analytic formula for on-shell element of K-matrix.
print(is_this_zero==0)

True


### Extracting coefficients of on-shell Kmatrix formula

In [26]:
# Extracting all coefficients for supplemental material
N,D = K_on_woodbury.as_numer_denom()

# fix LEC-independent part of denominator to be -1
should_be_min1 = D.subs({c1_symb:0, c2_symb:0, c3_symb:0})
D=D/(-should_be_min1)
N=N/(-should_be_min1)

K0 = N.subs({c1_symb:0, c2_symb:0, c3_symb:0})/D.subs({c1_symb:0, c2_symb:0, c3_symb:0})
Np = sympy.expand(N - K0*D)

# numerator
a1  = Np.coeff(c2_symb,2)
a2  = Np.coeff(c2_symb,1)
a3  = Np.coeff(c1_symb,1).subs({c3_symb:0})
a4  = Np.coeff(c1_symb,1).coeff(c3_symb,1)
a5  = Np.coeff(c3_symb,1).subs({c1_symb:0})
# denominator
a6  = D.coeff(c2_symb,2)
a7  = D.coeff(c2_symb,1)
a8  = D.coeff(c1_symb,1).subs({c3_symb:0})
a9  = D.coeff(c1_symb,1).coeff(c3_symb,1)
a10 = D.coeff(c3_symb,1).subs({c1_symb:0})

digits=5
print('K0',round(K0,digits))
print('a1',round(a1,digits))
print('a2',round(a2,digits))
print('a3',round(a3,digits))
print('a4',round(a4,digits))
print('a5',round(a5,digits))
print('a6',round(a6,digits))
print('a7',round(a7,digits))
print('a8',round(a8,digits))
print('a9',round(a9,digits))
print('a10',round(a10,digits))

print("$"+str(round(K0,digits))+"$ & $"+str(round(a1,digits))+"$ & $"+str(round(a2,digits))+"$ & $"+str(round(a3,digits))+"$ & $"+str(round(a4,digits))+"$ & $"+str(round(a5,digits))+"$ & $"+str(round(a6,digits))+"$ & $"+str(round(a7,digits))+"$ & $"+str(round(a8,digits))+"$ & $"+str(round(a9,digits))+"$ & $"+str(round(a10,digits))+"$")

K0 -1.58708
a1 7.74598
a2 -2.72226
a3 -60.58019
a4 -7.74598
a5 -0.03058
a6 1.05010
a7 -2.11053
a8 -12.81704
a9 -1.05010
a10 -0.16881
$-1.58708$ & $7.74598$ & $-2.72226$ & $-60.58019$ & $-7.74598$ & $-0.03058$ & $1.05010$ & $-2.11053$ & $-12.81704$ & $-1.05010$ & $-0.16881$


In [27]:
#lambdify K_on for faster evaluation 
K_on_formula=K0+\
    (a1*c2_symb**2+a2*c2_symb+a3*c1_symb+a4*c1_symb*c3_symb+a5*c3_symb)/ \
    (a6*c2_symb**2+a7*c2_symb+a8*c1_symb+a9*c1_symb*c3_symb+a10*c3_symb-1)

K_on_formula_func = sympy.lambdify([c1_symb, c2_symb, c3_symb], K_on_formula,
                                   modules='numpy')

#Finally calculate all on-shell K-matrix elements with above formula
c1_, c2_, c3_ = np.meshgrid(all_c1, all_c2, all_c3, indexing='ij')
c1_ = c1_.flatten()
c2_ = c2_.flatten()
c3_ = c3_.flatten()

all_onshell_K_viaprojection = K_on_formula_func(c1_, c2_, c3_)

In [28]:
# check if we can reproduce the on-shell K-matrix elements using above formula
print(np.allclose(all_onshell_K_viaprojection, all_halfshell_K[:,-1]))

True
